## HERMES - furosemide phenotype

#### Clone HeRmes repository

In [5]:
#system("git clone https://github.com/nicksunderland/heRmes.git")

#### Pushing changes to Github

In [6]:
# in ther terminal run
# cd /opt/notebooks
# dx download git_push.sh
# run bash git_push.sh 

#### Project & record ID

In [4]:
projectid <- "project-GvZyZ20J81vgPJGbJy8pgpyq"
recordid  <- "record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg"

#### Libraries

In [ ]:
library(glue)
library(data.table)
if (!requireNamespace("bit64", quietly = TRUE)) {
  install.packages("bit64")
}
library(bit64)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bit’




## Extract GP data

#### Download data dictionary

In [6]:
setwd("/opt/notebooks")
dataset <- glue("{projectid}:{recordid}")
cmd <- glue("dx extract_dataset {dataset} -ddd")
system(cmd)
dict_files <- list.files(pattern="codings|data_dictionary|entity_dictionary")
data_dict_file <- dict_files[grepl("data_dictionary", dict_files)]

#### Data dictionary filter function

In [7]:
#' @title filter_data_dict
#'
#' @param dict_path, str, path to the dataset.data_dictionary.csv
#' @param codes_str, list, list of lists representing UKBB column name, table entity, and search strategy list(name=, entity=, search=). 
#'   name must be a valid column name in the data_dictionary, entity a valid entity in the entity dictionary, and search either "matches"
#'   for exact matches, or starts with to match cases of multiple instances (repeated measures usually)
#'
#' @returns a filtered subset of the data_dictionary 
#'
filter_data_dict <- function(dict_path, codes_struc) {
    
    data_dict <- fread(dict_path)
    
    d <- lapply(codes_struc, function(x) {
        
        d0 <- data.table()
        if (x$search=="matches") {
            d0 <- data_dict[entity==x$entity & name==x$name]
        } else if (x$search=="startswith") {
            d0 <- data_dict[entity==x$entity & grepl(paste0("^", x$name), name)]
        }
        
        if (nrow(d0)==0) {
            cat(glue("Code [{x$name}] not found in data dictionary\n"))
            stop("Code not found error")
        }
        
        d0
        
    }) |> rbindlist(idcol = "item")
    
    return(d)
}

#### Data extraction function

In [8]:
#' @title extract_data
#'
#' @param dataset, str, a valid dataset id - format "{projectid}:{recordid}" 
#' @param fields, str, vector of UK-BB format column names e.g. p31
#' @param entity, str, string of length one - the entity to extract from e.g. participants
#' @param output, str, the base name for the output file, no extension
#'
#' @returns NULL side effect is starting a table-exporter job which outputs the file to /hermes3_data directory in the RAP
#'
extract_data <- function(dataset, fields, entity, output) {
    
    field_str <- paste0('-ifield_names="', fields, '"', collapse=" ") 
    
    cmd <- glue(
      "dx run table-exporter ",
      "-idataset_or_cohort_or_dashboard={dataset} ",
      "-ioutput={output} ",
      "-ioutput_format=TSV ",
      "-iheader_style=FIELD-NAME ",
      "-icoding_option=RAW ",
      "{field_str} ",
      "-ientity={entity} ",
      "--destination hermes3_data/"
    )    

    o <- system(cmd, intern = TRUE)
    cat(o, sep = "\n")
}

#### Define GP clinical data

In [9]:
gp_clinical_codes= list(eid           = list(name="eid",           entity="gp_clinical", search="matches"),
                        data_provider = list(name="data_provider", entity="gp_clinical", search="matches"),
                        date          = list(name="event_dt",      entity="gp_clinical", search="matches"),
                        read_2        = list(name="read_2",        entity="gp_clinical", search="matches"),
                        read_3        = list(name="read_3",        entity="gp_clinical", search="matches"),
                        value1        = list(name="value1",        entity="gp_clinical", search="matches"),
                        value2        = list(name="value2",        entity="gp_clinical", search="matches"),
                        value3        = list(name="value3",        entity="gp_clinical", search="matches"))

gp_clinical_data_dict = filter_data_dict(data_dict_file, gp_clinical_codes)
head(gp_clinical_data_dict, 3)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,gp_clinical,eid,string,,,NA,NA,,,,,NA,participant:eid,many_to_one,Participant ID,
data_provider,gp_clinical,data_provider,string,,data_coding_626,NA,NA,,,,,NA,,,Data provider,
date,gp_clinical,event_dt,date,,data_coding_819,NA,NA,,,yes,,NA,,,Date clinical code was entered,


#### Define GP medications data

In [10]:
gp_medication_codes= list(eid           = list(name="eid",           entity="gp_scripts", search="matches"),
                          data_provider = list(name="data_provider", entity="gp_scripts", search="matches"),
                          date          = list(name="issue_date",    entity="gp_scripts", search="matches"),
                          read_2        = list(name="read_2",        entity="gp_scripts", search="matches"),
                          bnf_code      = list(name="bnf_code",      entity="gp_scripts", search="matches"),
                          dmd_code      = list(name="dmd_code",      entity="gp_scripts", search="matches"),
                          drug_name     = list(name="drug_name",     entity="gp_scripts", search="matches"),
                          quantity      = list(name="quantity",      entity="gp_scripts", search="matches"))

gp_medication_data_dict = filter_data_dict(data_dict_file, gp_medication_codes)
head(gp_medication_data_dict, 3)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,gp_scripts,eid,string,,,NA,NA,,,,,NA,participant:eid,many_to_one,Participant ID,
data_provider,gp_scripts,data_provider,string,,data_coding_626,NA,NA,,,,,NA,,,Data provider,
date,gp_scripts,issue_date,date,,data_coding_819,NA,NA,,,yes,,NA,,,Date prescription was issued,


#### Run Table-Exporter extraction

In [12]:
data_file_paths <- list(
    demog = "/mnt/project/hermes3_data/data_participant.tsv",
    self  = "/mnt/project/hermes3_data/data_selfreportedillness.tsv",
    hesin = "/mnt/project/hermes3_data/data_hesin.tsv",
    diag  = "/mnt/project/hermes3_data/data_hesin_diag.tsv",
    oper  = "/mnt/project/hermes3_data/data_hesin_oper.tsv",
    gp    = "/mnt/project/hermes3_data/data_gp_clinical.tsv",
    med   = "/mnt/project/hermes3_data/data_gp_medication.tsv"
)

if (!file.exists(data_file_paths$demog)) {
    extract_data(dataset=dataset, fields=participant_data_dict$name, entity="participant", output = "data_participant")
}
if (!file.exists(data_file_paths$self)) {
    extract_data(dataset=dataset, fields=self_rep_data_dict$name,    entity="participant", output = "data_selfreportedillness")
}
if (!file.exists(data_file_paths$hesin)) {
    extract_data(dataset=dataset, fields=hes_data_dict$name,         entity="hesin",       output = "data_hesin")
}
if (!file.exists(data_file_paths$diag)) {
    extract_data(dataset=dataset, fields=hesdiag_data_dict$name,     entity="hesin_diag",  output = "data_hesin_diag")
}
if (!file.exists(data_file_paths$oper)) {
    extract_data(dataset=dataset, fields=hesoper_data_dict$name,     entity="hesin_oper",  output = "data_hesin_oper")
}
if (!file.exists(data_file_paths$gp)) {
    extract_data(dataset=dataset, fields=gp_clinical_data_dict$name, entity="gp_clinical", output = "data_gp_clinical")
}
if (!file.exists(data_file_paths$med)) {
    extract_data(dataset=dataset, fields=gp_medication_data_dict$name, entity="gp_scripts", output = "data_gp_medication")
}

## Read in extracted data

In [13]:
data_files <- list()

for (i in seq_along(data_file_paths)) {
    if (!file.exists(data_file_paths[[i]])) {
        cat(glue("Error:\nFile {basename(data_file_paths[[i]])} not found, ",
                 "check the Monitor tab for the status of the Table-exporter ",
                 "and the 'hermes_data' folder. If this has finished try ",
                 "launching another Notebook session/instance (I'm not sure why ",
                 "the mounted /mnt/project/ file structure doesn't refresh when ",
                 "files are added externally."))
        stop("file not found error")
    }
    f <- data_file_paths[[i]]
    n <- names(data_file_paths)[i]
    cat(glue('...{n}: {f}\n'), sep="\n")
    flush.console()
    data_files[[n]] <- fread(f)
}

lapply(data_files, head, n = 5)

...demog: /mnt/project/hermes3_data/data_participant.tsv
...self: /mnt/project/hermes3_data/data_selfreportedillness.tsv
...hesin: /mnt/project/hermes3_data/data_hesin.tsv
...diag: /mnt/project/hermes3_data/data_hesin_diag.tsv
...oper: /mnt/project/hermes3_data/data_hesin_oper.tsv
...gp: /mnt/project/hermes3_data/data_gp_clinical.tsv
...med: /mnt/project/hermes3_data/data_gp_medication.tsv


Warning message in fread(f):
“Found and resolved improper quoting out-of-sample. First healed line 89832: <<5715998	3	2005-06-30		14.04.21.00.00		Typherix 25micrograms/0.5ml vaccine solution for injection pre-filled syringes (GlaxoSmithKline UK Ltd)	"1 - 0.5ml pre-filled syringe+ 23G 1\"& 25G 5/8\" needles">>. If the fields are not quoted (e.g. field separator does not appear within any field), try quote="" to avoid this warning.”
Warning message in require_bit64_if_needed(ans):
“Some columns are type 'integer64' but package bit64 is not installed. Those columns will print as strange looking floating point data. There is no need to reload the data. Simply install.packages('bit64') to obtain the integer64 print method and print the data again.”
Warning message in require_bit64_if_needed(x):
“Some columns are type 'integer64' but package bit64 is not installed. Those columns will print as strange looking floating point data. There is no need to reload the data. Simply install.packages('b

$demog
       eid  p190   p31 p21022 p21000_i0 p21000_i1 p21000_i2 p21000_i3 p22001
     <int> <int> <int>  <int>     <int>     <int>     <int>     <int>  <int>
1: 1062757    NA     0     62      1001        NA        NA        NA      0
2: 2217356    NA     1     57      1001        NA        NA        NA     NA
3: 3712401    NA     1     56      1001        NA        NA        NA      1
4: 1011090    NA     0     49      1001        NA        NA        NA      0
5: 2874739    NA     0     59      1001        NA        NA        NA      0
   p22006 p22009_a1 p22009_a2 p22009_a3 p22009_a4 p22009_a5
    <int>     <num>     <num>     <num>     <num>     <num>
1:      1  -12.6693   4.30928 -2.234380 -0.109226  -7.36400
2:     NA        NA        NA        NA        NA        NA
3:      1  -13.7558   5.64773 -3.581870  7.741650  21.02850
4:      1  -12.3794   2.03865 -0.837131 -0.562303   2.60283
5:      1  -12.6667   4.18819 -1.969960  2.858750   0.65970

$self
       eid p20002_i0_a0 p20002_i0_a1 p20002_i0_a2 p20002_i0_a3 p20002_i0_a4
     <int>        <int>        <int>        <int>        <int>        <int>
1: 1000074         1351           NA           NA           NA           NA
2: 1000194         1086           NA           NA           NA           NA
3: 1000258         1065         1265         1465           NA           NA
4: 1000280         1436           NA           NA           NA           NA
5: 1000299         1154           NA           NA           NA           NA
   p20002_i0_a5 p20002_i0_a6 p20002_i0_a7 p20002_i0_a8 p20002_i0_a9
          <int>        <int>        <int>        <int>        <int>
1:           NA           NA           NA           NA           NA
2:           NA           NA           NA           NA           NA
3:           NA           NA           NA           NA           NA
4:           NA           NA           NA           NA           NA
5:           NA           NA           NA           NA           NA
   p20002_i0_a10 p20002_i0_a11 p20002_i0_a12 p20002_i0_a13 p20002_i0_a14
           <int>         <int>         <int>         <int>         <int>
1:            NA            NA            NA            NA            NA
2:            NA            NA            NA            NA            NA
3:            NA            NA            NA            NA            NA
4:            NA            NA            NA            NA            NA
5:            NA            NA            NA            NA            NA
   p20002_i0_a15 p20002_i0_a16 p20002_i0_a17 p20002_i0_a18 p20002_i0_a19
           <int>         <int>         <int>         <int>         <int>
1:            NA            NA            NA            NA            NA
2:            NA            NA            NA            NA            NA
3:            NA            NA            NA            NA            NA
4:            NA            NA            NA            NA            NA
5:            NA            NA            NA            NA            NA
   p20002_i0_a20 p20002_i0_a21 p20002_i0_a22 p20002_i0_a23 p20002_i0_a24
           <int>         <int>         <int>         <int>         <int>
1:            NA            NA            NA            NA            NA
2:            NA            NA            NA            NA            NA
3:            NA            NA            NA            NA            NA
4:            NA            NA            NA            NA            NA
5:            NA            NA            NA            NA            NA
   p20002_i0_a25 p20002_i0_a26 p20002_i0_a27 p20002_i0_a28 p20002_i0_a29
           <int>         <int>         <int>         <int>        <lgcl>
1:            NA            NA            NA            NA            NA
2:            NA            NA            NA            NA            NA
3:            NA            NA            NA            NA            NA
4:            NA            NA            NA            NA            NA
5:            NA            NA            NA    

## Rename columns

In [ ]:
rename_cols <- function(d, code_struc) {
    for (col in names(code_struc)) {
        if (code_struc[[col]]$search=="matches") {
            setnames(d, code_struc[[col]]$name, col)
        } else if (code_struc[[col]]$search=="startswith") {
            regex     <- paste0("^", code_struc[[col]]$name)
            matches   <- names(d)[grepl(regex, names(d))]
            new_names <- paste0(col, "_", 1:length(matches))
            setnames(d, matches, new_names)
        }
    }
    return(d)
}

data_files$demog <- rename_cols(data_files$demog, code_struc=participant_codes)
data_files$self  <- rename_cols(data_files$self,  code_struc=self_illness_codes)
data_files$hesin <- rename_cols(data_files$hesin, code_struc=hesin_to_extract)
data_files$diag  <- rename_cols(data_files$diag,  code_struc=hesdiag_to_extract)
data_files$oper  <- rename_cols(data_files$oper,  code_struc=hesproc_to_extract)
data_files$gp    <- rename_cols(data_files$gp,    code_struc=gp_clinical_codes)
data_files$med   <- rename_cols(data_files$med,   code_struc=gp_medication_codes)

lapply(data_files, head, n = 1)

## Data processing

In [ ]:
codes <- fread(file.path("heRmes", "inst", "extdata", "hermes_furosemide_codes", "hermes_furosemide_codes.tsv"))
codes[, code := sub("^'(.+?)'$", code)]
codes <- codes[!is.na(code_type)] 
head(codes)